In [ ]:
# Import the X-HEEP Python class
from pynq import x_heep

In [ ]:
# Load the X-HEEP bitstream
x_heep = x_heep()

In [ ]:
# Compile the application
x_heep.compile_app("virtual_adc_read")

In [ ]:
# Init the ADC memory
adc = x_heep.init_adc_mem()

In [ ]:
# Reset the ADC memory
x_heep.reset_adc_mem(adc)

In [ ]:
# Write the ADC memory
x_heep.write_adc_mem(adc)

In [ ]:
# Run the application
x_heep.run_app()

In [ ]:
# Delete adc
del adc

In [ ]:
# Verify the output
file_expected = open("/home/xilinx/x-heep-femu-sdk/sw/riscv/build/adc_in.bin", mode="rb")
expected_byte = file_expected.read()
file_stdout = open("/home/xilinx/x-heep-femu-sdk/sw/riscv/build/stdout.txt", "r")
stdout_byte = file_stdout.readlines()

allEqual = True
for i in range(int(len(expected_byte)/4)):
    written = int((expected_byte[i*4] << 24) | (expected_byte[i*4+1] << 16) | (expected_byte[i*4+2] << 8) | expected_byte[i*4+3])
    read = int(stdout_byte[2*i].strip().split(" ")[1], 16)
    if written != read:
        allEqual = False
        break
        
if allEqual:
    print("Test Passed!")
else:
    print("Test Failed!")